In [ ]:
import gensim
print(gensim.__version__)

4.2.0


In [ ]:
import nltk
#nltk.download("stopwords")
#nltk.download('wordnet')
#nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
#!pip install --upgrade gensim

In [ ]:
import pandas as pd
import numpy as np
import ssl
import gensim
from gensim.models.callbacks import CallbackAny2Vec

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
train=pd.read_csv('/content/suggest_train (1).csv')
train.head()

,item_id,title,category_id,name
0,335330,Раствор. Товарный бетон от производителя,15,Для дома и дачи|Ремонт и строительство|Стройма...
1,137480,Детская кроватка,16,"Для дома и дачи|Мебель и интерьер|Кровати, див..."
2,242690,"Кровать -качалка с ящиком для вещей, два полож...",34,Личные вещи|Товары для детей и игрушки|Детская...
3,434016,Линолеум комитекс лин печора орфей 431,15,Для дома и дачи|Ремонт и строительство|Стройма...
4,237890,Распродажа имущества предприятия. Компьютеры,7,Бытовая электроника|Настольные компьютеры


**Linear model (just to see how it goes)** 

In [ ]:
train_titles, val_titles, y_train, y_val = train_test_split(
    train['title'],
    train['category_id'],
    random_state=1
)

In [ ]:
train_titles_clean = train_titles.apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True))
val_titles_clean = val_titles.apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True))
train_titles_clean

18960                                бильярдный стол
11353                крассула оватта денежное дерево
7449                                фиксатор вальгус
14308                               подставка цветов
11889                          комплект мебели ибица
                            ...                     
10955                         мебель детской комнаты
17289                             supertooth disco 2
5192                             ноут 4 32гб золотой
12172    электронная фигурка человекапаука свет звук
235                 спасалки зимней рыбалки свистком
Name: title, Length: 15000, dtype: object

In [ ]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords):
    ## clean (convert to lowercase and remove punctuations and  characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
count_vectorizer = CountVectorizer()
count_vectorizer

CountVectorizer()

In [ ]:
X_train = count_vectorizer.fit_transform(train_titles.values)
X_val = count_vectorizer.transform(val_titles.values)

In [ ]:
log_reg = LogisticRegression(multi_class='auto', solver='lbfgs', max_iter=50000)
log_reg.fit(X_train, y_train)

LogisticRegression(max_iter=50000)

In [ ]:
what = log_reg.predict_proba(X_val)
for_acc3 = [i.argsort()[-3:][::-1] for i in what]
#pd.DataFrame(for_acc3).to_csv('solution.csv', header=['top1', 'top2', 'top3'], index=False)

In [ ]:
what[0].argsort()

array([ 0, 11, 35,  5, 10,  3,  6, 33, 25,  2, 45,  7, 39, 16, 36, 40,  1,
       50, 14, 13, 26, 22, 43, 18, 12, 32, 52, 48, 38, 29, 31, 19,  9, 27,
       30, 44, 42, 34, 24, 15, 23, 20, 17, 41, 21, 51, 28, 53, 49, 37, 46,
       47,  4,  8])

In [ ]:
def accuracy(list3, list_ans):
  list3 = list(list3)
  list_ans = list(list_ans)
  right = 0
  for i in range(len(list_ans)):
    if list_ans[i] in list3[i]:
      right += 1
  print(right/len(list_ans))

In [ ]:
accuracy(for_acc3, y_val)

0.8584


In [ ]:
predslogreg = log_reg.predict(X_val)
accuracy_score(predslogreg, y_val)

0.739

**w2v model**

In [ ]:
class LossLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss
        

class EpochLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        print(f'Epoch {self.epoch}')
        self.epoch += 1


In [ ]:
import re
from gensim.models.word2vec import Word2Vec
from sklearn.linear_model import SGDClassifier

WORD_PATTERN = '(?u)\\b\\w\\w+\\b'

reg_exp = re.compile(pattern=WORD_PATTERN)
w2v_train_data = pd.read_csv('/content/suggest_train (1).csv',  index_col=0)
w2v_train_data.head(2)

,title,category_id,name
item_id,,,
335330,Раствор. Товарный бетон от производителя,15,Для дома и дачи|Ремонт и строительство|Стройма...
137480,Детская кроватка,16,"Для дома и дачи|Мебель и интерьер|Кровати, див..."


In [ ]:
train_titles, val_titles, y_train, y_val = train_test_split(
    w2v_train_data['title'],
    w2v_train_data['category_id'],
    random_state=1
)

In [ ]:
sentences = [reg_exp.findall(s.lower()) for s in w2v_train_data.title]

In [ ]:
w2v_model = Word2Vec(sg=1,)
w2v_model.build_vocab(sentences)
w2v_model.train(
    sentences,
    total_examples=w2v_model.corpus_count,
    epochs=174,
    compute_loss=True,
    callbacks=[LossLogger()]
)

Loss after epoch 0: 130211.8984375
Loss after epoch 1: 116634.8671875
Loss after epoch 2: 93111.046875
Loss after epoch 3: 77665.5625
Loss after epoch 4: 74219.15625
Loss after epoch 5: 51096.46875
Loss after epoch 6: 42304.6875
Loss after epoch 7: 58883.0625
Loss after epoch 8: 54180.8125
Loss after epoch 9: 50560.1875
Loss after epoch 10: 48148.5625
Loss after epoch 11: 45763.5625
Loss after epoch 12: 44223.75
Loss after epoch 13: 42246.125
Loss after epoch 14: 40452.9375
Loss after epoch 15: 30036.5
Loss after epoch 16: 25735.3125
Loss after epoch 17: 35472.625
Loss after epoch 18: 22972.25
Loss after epoch 19: 31428.125
Loss after epoch 20: 31977.125
Loss after epoch 21: 30071.25
Loss after epoch 22: 30210.875
Loss after epoch 23: 28915.875
Loss after epoch 24: 28623.625
Loss after epoch 25: 29938.125
Loss after epoch 26: 27174.625
Loss after epoch 27: 26007.25
Loss after epoch 28: 25901.75
Loss after epoch 29: 24927.875
Loss after epoch 30: 16857.0
Loss after epoch 31: 18556.875
L

(7293320, 12682686)

In [ ]:
class Word2VecTransformer:
    
    def __init__(self, w2v_model, word_pattern):
        
        self.w2v_model = w2v_model
        self.word_pattern = word_pattern
        self.re = re.compile(pattern=self.word_pattern)
        
    def fit(self, X):
        return self
    
    def transform(self, X):
        
        X_transformed = np.zeros((len(X), self.w2v_model.wv.vector_size))
        for i, title in enumerate(X):
            
            title_vector = np.zeros((self.w2v_model.wv.vector_size,))
            tokens = self.re.findall(title.lower())
            for token in tokens:
                if token in self.w2v_model.wv.key_to_index:
                    title_vector += self.w2v_model.wv.get_vector(token)
                    
            X_transformed[i] = title_vector
        
        return X_transformed

In [ ]:
w2v_model.wv.similar_by_word('продам')

[('женскую', 0.4387795031070709),
 ('норковую', 0.42326101660728455),
 ('породам', 0.4063822031021118),
 ('шубу', 0.39937251806259155),
 ('сумку', 0.3629215359687805),
 ('детскую', 0.3532012104988098),
 ('абсолютно', 0.349235475063324),
 ('новую', 0.34586668014526367),
 ('машинку', 0.3416174352169037),
 ('машину', 0.30728036165237427)]

In [ ]:
!pip install --upgrade gensim
import gensim
print(gensim.__version__)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.6 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


3.6.0


In [ ]:
w2v_transformer = Word2VecTransformer(w2v_model=w2v_model, word_pattern=WORD_PATTERN)

train_w2v = w2v_transformer.transform(train_titles.values)
val_w2v = w2v_transformer.transform(val_titles.values)

In [ ]:
log_reg = SGDClassifier(loss='hinge',class_weight='balanced')
clf =log_reg.fit(train_w2v, y_train)
calibrator = CalibratedClassifierCV(clf, cv='prefit')
model=calibrator.fit(train_w2v, y_train)

model.fit(train_w2v, y_train)

CalibratedClassifierCV(base_estimator=SGDClassifier(class_weight='balanced'),
                       cv='prefit')

In [ ]:
def accuracy(list3, list_ans):
  list3 = list(list3)
  list_ans = list(list_ans)
  right = 0
  for i in range(len(list_ans)):
    if list_ans[i] in list3[i]:
      right += 1
  print(right/len(list_ans))

In [ ]:
what = model.predict_proba(val_w2v)
for_acc3 = [i.argsort()[-3:][::-1] for i in what]
#pd.DataFrame(for_acc3).to_csv('solution.csv', header=['top1'

In [ ]:
accuracy(for_acc3, y_val)

0.752


In [ ]:
preds = log_reg.predict(val_w2v)
accuracy_score(preds, y_val)

0.6322

**Trying something more advanced**

In [ ]:
#!pip install lime
#!pip install transformers
#!pip install tensorflow

In [ ]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

In [ ]:
WORD_PATTERN = '(?u)\\b\\w\\w+\\b'

reg_exp = re.compile(pattern=WORD_PATTERN)
w2v_train_data = pd.read_csv('/content/suggest_train (1).csv',  index_col=0)
print(w2v_train_data.shape)
w2v_train_data.head()

(20000, 3)


,title,category_id,name
item_id,,,
335330,Раствор. Товарный бетон от производителя,15,Для дома и дачи|Ремонт и строительство|Стройма...
137480,Детская кроватка,16,"Для дома и дачи|Мебель и интерьер|Кровати, див..."
242690,"Кровать -качалка с ящиком для вещей, два полож...",34,Личные вещи|Товары для детей и игрушки|Детская...
434016,Линолеум комитекс лин печора орфей 431,15,Для дома и дачи|Ремонт и строительство|Стройма...
237890,Распродажа имущества предприятия. Компьютеры,7,Бытовая электроника|Настольные компьютеры


In [ ]:
## split dataset
dtf_train, dtf_test = model_selection.train_test_split(w2v_train_data, test_size=0.3)
## get target
y_train = dtf_train["category_id"].values
y_test = dtf_test["category_id"].values

In [ ]:
#const stopwords = require('stopwords-ru')
from nltk.corpus import stopwords
lst_stopwords = nltk.corpus.stopwords.words("russian")

LookupError: ignored

In [ ]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords):
    ## clean (convert to lowercase and remove punctuations and  characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [ ]:
dtf_train["text_clean"] = dtf_train["title"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True))
dtf_test["text_clean"] = dtf_test["title"].apply(lambda x: 
          utils_preprocess_text(x, flg_stemm=False, flg_lemm=True))
dtf_train.head()

,title,category_id,name,text_clean
item_id,,,,
277990,Люстра подвесная,19,Для дома и дачи|Мебель и интерьер|Предметы инт...,люстра подвесная
14056,Пара колонок denon SC-N9,12,"Бытовая электроника|Аудио и видео|Акустика, ко...",пара колонок denon scn9
362689,Картина из страз,23,Для дома и дачи|Мебель и интерьер|Другое,картина страз
42160,Дутики rubber duck оригинал из Америки новые в...,39,"Личные вещи|Одежда, обувь, аксессуары|Женская ...",дутики rubber duck оригинал америки новые р
343299,Обруч хулахуп vita health hoop,46,Хобби и отдых|Спорт и отдых|Фитнес и тренажёры,обруч хулахуп vita health hoop


In [ ]:
## Tf-Idf (advanced variant of BoW)
vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))

corpus = dtf_train["text_clean"]
vectorizer.fit(corpus)
X_train = vectorizer.transform(corpus)
dic_vocabulary = vectorizer.vocabulary_

In [ ]:
classifier = naive_bayes.MultinomialNB()
## pipeline
model = pipeline.Pipeline([("vectorizer", vectorizer),  
                           ("classifier", classifier)])
## train classifier
model["classifier"].fit(X_train, y_train)
## test
X_test = dtf_test["text_clean"].values
#predicted = model.predict(X_test)
#predicted_prob = model.predict_proba(X_test)
#predicted

In [ ]:
what = model.predict(X_test)
for_acc3 = [i.argsort()[-3:][::-1] for i in what]
#pd.DataFrame(for_acc3).to_csv('solution.csv', header=['top1', 'top2', 'top3'], index=False)

In [ ]:
accuracy(for_acc3, y_test)

0.0185


In [ ]:
1 - accuracy_score(for_acc3, y_test)

0.9845

**Code from StackOverflow**

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import SGDClassifier

import pandas as pd
import numpy as np

#df = pd.read_csv('./smsspamcollection//SMSSpamCollection', sep='\t', names=["label", "message"])
#df['label'][df['label']=='ham'] = np.random.choice(['hamA','hamB'],np.sum(df['label']=='ham'))
#X_train = df['message']
#y_train = df['label']

In [ ]:
X_train

<15000x17203 sparse matrix of type '<class 'numpy.int64'>'
	with 54537 stored elements in Compressed Sparse Row format>

In [ ]:
y_train.shape

(15000,)

In [ ]:
cv = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

pipeline_sgd = Pipeline([
     ('vect', CountVectorizer()),
     ('tfdif', TfidfTransformer()),
     ('nb', CalibratedClassifierCV(base_estimator = SGDClassifier(), cv=cv)),
])

model = pipeline_sgd.fit(list[X_train], list[y_train])

TypeError: ignored

In [ ]:
n_top_labels = 3
probas = model.predict_proba(X_train[:5])
top_n_lables_idx = np.argsort(-probas)
top_n_probs = np.round(-np.sort(-probas),3)
top_n_labels = [model.classes_[i] for i in top_n_lables_idx]

results = list(zip(top_n_labels, top_n_probs))

pd.DataFrame(results)


In [ ]:
dataset=pd.read_csv('/content/suggest_test (1).csv')
dataset.head()

,title
0,Телевизор Samsung
1,"Игрушки ручной работы (вязание, валяние)"
2,Раковина с ножкой и смесителем
3,Куколки mattel
4,Автомобильный сабвуфер активный Kicx ICQ-300BA
